**Modulo 2: Python para Machine Learning**
* Instructor: [Juan Maniglia](https://juanmaniglia.github.io)

# Parte 2.1: Introducción to Pandas

[Pandas](http://pandas.pydata.org/) es una biblioteca de código abierto que proporciona estructuras de datos y herramientas de análisis de datos fáciles de usar y de alto rendimiento para el lenguaje de programación Python.

El siguiente código carga el conjunto de datos MPG en un data frame:

In [1]:
# Simple dataframe
import os
import pandas as pd

df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/auto-mpg.csv")
print(df[0:5])

    mpg  cylinders  displacement horsepower  weight  acceleration  year  \
0  18.0          8         307.0        130    3504          12.0    70   
1  15.0          8         350.0        165    3693          11.5    70   
2  18.0          8         318.0        150    3436          11.0    70   
3  16.0          8         304.0        150    3433          12.0    70   
4  17.0          8         302.0        140    3449          10.5    70   

   origin                       name  
0       1  chevrolet chevelle malibu  
1       1          buick skylark 320  
2       1         plymouth satellite  
3       1              amc rebel sst  
4       1                ford torino  


La función **display** proporciona una visualización más limpia que simplemente imprimir.

In [2]:
pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 5)
display(df)

,mpg,cylinders,displacement,...,year,origin,name
0,18.0,8,307.0,...,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,...,70,1,buick skylark 320
...,...,...,...,...,...,...,...
396,28.0,4,120.0,...,82,1,ford ranger
397,31.0,4,119.0,...,82,1,chevy s-10


Es posible generar un segundo data frame para mostrar información estadística sobre el primero data frame.

In [3]:
# Eliminar los no numéricos
df = df.select_dtypes(include=['int', 'float'])

headers = list(df.columns.values)
fields = []

for field in headers:
    fields.append({
        'name' : field,
        'mean': df[field].mean(),
        'var': df[field].var(),
        'sdev': df[field].std()
    })

for field in fields:
    print(field)

{'name': 'mpg', 'mean': 23.514572864321607, 'var': 61.089610774274405, 'sdev': 7.815984312565782}
{'name': 'cylinders', 'mean': 5.454773869346734, 'var': 2.893415439920003, 'sdev': 1.7010042445332119}
{'name': 'displacement', 'mean': 193.42587939698493, 'var': 10872.199152247384, 'sdev': 104.26983817119591}
{'name': 'weight', 'mean': 2970.424623115578, 'var': 717140.9905256763, 'sdev': 846.8417741973268}
{'name': 'acceleration', 'mean': 15.568090452261307, 'var': 7.604848233611383, 'sdev': 2.757688929812676}
{'name': 'year', 'mean': 76.01005025125629, 'var': 13.672442818627143, 'sdev': 3.697626646732623}
{'name': 'origin', 'mean': 1.5728643216080402, 'var': 0.6432920268850549, 'sdev': 0.8020548777266148}


Este código genera una lista de diccionarios que contienen esta información estadística. Esta información es similar al código JSON,  Para que sea un JSON adecuado, el programa debe agregar estos registros a una lista y llamar al comando **dumps** de la biblioteca JSON de Python llamado.

El programa Python puede convertir esta información similar a JSON en un data frame para una mejor visualización

In [4]:
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 0)
df2 = pd.DataFrame(fields)
display(df2)

,name,mean,var,sdev
0,mpg,23.514573,61.089611,7.815984
1,cylinders,5.454774,2.893415,1.701004
2,displacement,193.425879,10872.199152,104.269838
3,weight,2970.424623,717140.990526,846.841774
4,acceleration,15.568090,7.604848,2.757689
5,year,76.010050,13.672443,3.697627
6,origin,1.572864,0.643292,0.802055


## Missing Values

Missing values son una realidad del aprendizaje automático. Idealmente, cada fila de datos tendrá valores para todas las columnas. Sin embargo, esto no suele ser el caso. La mayoría de los valores están presentes en la base de datos MPG. Sin embargo, faltan valores en la columna de caballos de fuerza. Una práctica común es reemplazar los valores faltantes con el valor medio de esa columna.

In [6]:
import os
import pandas as pd

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])
print(f"horsepower tiene na? {pd.isnull(df['horsepower']).values.any()}")
    
print("Rellenar missing values...")
med = df['horsepower'].median()
df['horsepower'] = df['horsepower'].fillna(med)
# df = df.dropna() # you can also simply drop NA values
                 
print(f"horsepower tiene na? {pd.isnull(df['horsepower']).values.any()}")

horsepower tiene na? True
Rellenar missing values...
horsepower tiene na? False


# Dealing with Outliers

Outliers son valores que son inusualmente altos o bajos. A veces, los valores atípicos son simplemente errores;  Outliers también pueden ser valores verdaderamente grandes o pequeños que pueden ser difíciles de abordar.Por lo general, consideramos que los valores atípicos son un valor que se encuentra a varias desviaciones estándar de la media. La siguiente función puede eliminar dichos valores.

In [7]:
# Eliminar todas las filas donde la columna especificada es +/- desviaciones estándar sd
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean())
                          >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)

El siguiente código eliminará todas las filas del conjunto de datos Auto MPG donde la potencia es más de dos desviaciones estándar por encima o por debajo de la media.

In [15]:
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore
from ipykernel import kernelapp as app

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA','?'])

med = df['horsepower'].median()
df['horsepower'] = df['horsepower'].fillna(med)

# eliminar la columna name
df.drop(columns='name',axis=1,inplace=True)

# Eliminar outliers en horsepower
print("Longitud antes de eliminar los Outliers de MPG: {}".format(len(df)))
remove_outliers(df,'mpg',2)
print("Longitud despues de eliminar los Outliers de MPG: {}".format(len(df)))

pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 10)
display(df)

Longitud antes de eliminar los Outliers de MPG: 398
Longitud despues de eliminar los Outliers de MPG: 388


,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
0,18.0,8,307.0,130.0,3504,12.0,70,1
1,15.0,8,350.0,165.0,3693,11.5,70,1
2,18.0,8,318.0,150.0,3436,11.0,70,1
3,16.0,8,304.0,150.0,3433,12.0,70,1
4,17.0,8,302.0,140.0,3449,10.5,70,1
...,...,...,...,...,...,...,...,...
392,27.0,4,151.0,90.0,2950,17.3,82,1
393,27.0,4,140.0,86.0,2790,15.6,82,1
395,32.0,4,135.0,84.0,2295,11.6,82,1
396,28.0,4,120.0,79.0,2625,18.6,82,1


## Dropping Fields

Algunos campos que no tienen ningún valor para la red neuronal deben eliminarse. El siguiente código elimina la columna de nombre del conjunto de datos MPG.

In [14]:
import os
import pandas as pd

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA','?'])

print(f"Antes de eliminar: {list(df.columns)}")
df.drop(columns='name',axis= 1, inplace=True)
print(f"Después de eliminar: {list(df.columns)}")

Antes de eliminar: ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin', 'name']
Después de eliminar: ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']


## Concatenando Rows and Columns
Python puede concatenar filas y columnas para formar nuevos data frame. El siguiente código crea un nuevo data frame a partir de las columnas **nombre** y **caballos de fuerza** del conjunto de datos Auto MPG.  

In [16]:
# Crear un nuevo dataframe para name y horsepower

import os
import pandas as pd

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA','?'])

col_horsepower = df['horsepower']
col_name = df['name']
result = pd.concat([col_name, col_horsepower], axis=1)

pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 5)
display(result)

,name,horsepower
0,chevrolet chevelle malibu,130.0
1,buick skylark 320,165.0
...,...,...
396,ford ranger,79.0
397,chevy s-10,82.0


La función **concat** también puede concatenar dos filas juntas. Este código concatena las dos primeras filas y las dos últimas filas del conjunto de datos Auto MPG.

In [17]:
# Crea un nuevo dataframe para las primeras 2 filas y las últimas 2 filas

import os
import pandas as pd

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA','?'])

result = pd.concat([df[0:2],df[-2:]], axis=0)

pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 0)
display(result)

,mpg,cylinders,displacement,...,year,origin,name
0,18.0,8,307.0,...,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,...,70,1,buick skylark 320
396,28.0,4,120.0,...,82,1,ford ranger
397,31.0,4,119.0,...,82,1,chevy s-10


## Training and Validation

Debemos evaluar un modelo de Machine_Learning en función de su capacidad para predecir datos que nunca antes había visto. Debido a esto, a menudo dividimos los datos de entrenamiento en un conjunto de validación y entrenamiento.

* **Training Data** - **In Sample Data** - Los datos que la red neuronal usa para entrenar. 
* **Validation Data** - **Out of Sample Data** - Los datos sobre los que se evalúa el modelo Machine Learning después de ajustarse a los datos de entrenamiento.

Existen dos medios efectivos para manejar los datos de entrenamiento y validación:

* **Training/Validation Split** - El programa divide los datos de acuerdo con alguna proporción entre un conjunto de entrenamiento y validación. Las tasas típicas son 80% de capacitación y 20% de validación.
* **K-Fold Cross Validation** - El programa divide los datos en varios pliegues y modelos. Debido a que el programa crea la misma cantidad de modelos que pliegues, el programa puede generar predicciones fuera de la muestra para todo el conjunto de datos.

**Figure 2.TRN-VAL: Training and Validation**
![Training and Validation](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/class_1_train_val.png "Training and Validation")


In [18]:
import os
import pandas as pd
import numpy as np

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA','?'])

# Suele ser una buena idea revolver
df = df.reindex(np.random.permutation(df.index)) 

mask = np.random.rand(len(df)) < 0.8
trainDF = pd.DataFrame(df[mask])
validationDF = pd.DataFrame(df[~mask])

print(f"Training DF: {len(trainDF)}")
print(f"Validation DF: {len(validationDF)}")

Training DF: 321
Validation DF: 77


### Convirtiendo un Dataframe a una Matriz

Las redes neuronales no operan directamente en Python data frames. Una red neuronal requiere una matriz numérica. El programa utiliza la propiedad **valores** de un data frame para convertir los datos en una matriz.

In [19]:
df.values

array([[25.0, 4, 90.0, ..., 75, 2, 'volkswagen dasher'],
       [19.0, 6, 232.0, ..., 71, 1, 'amc gremlin'],
       [17.0, 8, 305.0, ..., 79, 1, 'chevrolet caprice classic'],
       ...,
       [16.2, 6, 163.0, ..., 78, 2, 'peugeot 604sl'],
       [39.1, 4, 79.0, ..., 81, 3, 'toyota starlet'],
       [17.0, 8, 260.0, ..., 77, 1, 'oldsmobile cutlass supreme']],
      dtype=object)

Es posible que desee convertir solo algunas de las columnas, para omitir la columna de nombre, use el siguiente código.

In [20]:
df[['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values

array([[ 25. ,   4. ,  90. , ...,  16.5,  75. ,   2. ],
       [ 19. ,   6. , 232. , ...,  13. ,  71. ,   1. ],
       [ 17. ,   8. , 305. , ...,  15.4,  79. ,   1. ],
       ...,
       [ 16.2,   6. , 163. , ...,  15.8,  78. ,   2. ],
       [ 39.1,   4. ,  79. , ...,  16.9,  81. ,   3. ],
       [ 17. ,   8. , 260. , ...,  19. ,  77. ,   1. ]])

## Guardar un Dataframe en CSV

In [22]:
import os
import pandas as pd
import numpy as np

path = "data"

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA','?'])

filename_write = os.path.join(path, "auto-mpg-shuffle.csv")
df = df.reindex(np.random.permutation(df.index))
# Specify index = false to not write row numbers
df.to_csv(filename_write, index=False) 
print("Done")

Done


## Guardar un Dataframe en Pickle

Una variedad de programas de software pueden hacer uso de archivos de texto almacenados como CSV. Sin embargo, tardan más en generarse y, a veces, pueden perder pequeñas cantidades de precisión en la conversión.  Otro formato es [Pickle](https://docs.python.org/3/library/pickle.html).  En general, la salida será CSV porque es muy compatible, incluso fuera de Python. El siguiente código almacena el marco de datos en Pickle.

In [23]:
import os
import pandas as pd
import numpy as np
import pickle

path = "data"

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA','?'])

filename_write = os.path.join(path, "auto-mpg-shuffle.pkl")
df = df.reindex(np.random.permutation(df.index))

with open(filename_write,"wb") as fp:
    pickle.dump(df, fp)

La carga del archivo pickle nuevamente en la memoria se logra mediante las siguientes líneas de código.

In [24]:
import os
import pandas as pd
import numpy as np
import pickle

path = "data"

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA','?'])

filename_read = os.path.join(path, "auto-mpg-shuffle.pkl")

with open(filename_write,"rb") as fp:
    df = pickle.load(fp)

pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 5)
display(df)

,mpg,cylinders,displacement,...,year,origin,name
394,44.0,4,97.0,...,82,2,vw pickup
353,33.0,4,105.0,...,81,2,volkswagen jetta
...,...,...,...,...,...,...,...
172,25.0,4,90.0,...,75,2,volkswagen dasher
9,15.0,8,390.0,...,70,1,amc ambassador dpl
